In [4]:
!pip install --upgrade azureml-sdk azureml-widgets

^C


In [6]:
!pip install azureml-widgets

In [5]:
!pip install azureml-sdk

ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->azureml-widgets) (0.2.5)
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10507 sha256=6a74c06384c361b10858aa94565f2b0a9ed59fa606c09a49a131e016ac6e6002
  Stored in directory: c:\users\kelvi\appdata\local\pip\cache\wheels\7f\41\10\f70b83a1164fdb95e7bc37bace13114a024227e56c2fee02bb
Successfully built fusepy
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directo

In [7]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config('config.json')
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Performing interactive authentication. Please follow the instructions on the terminal.
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Ready to use Azure ML 1.19.0 to work with dp100-WS


In [14]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False


In [4]:
!dir

kaggle.json  t-e70fc5cb-a0cf-4f61-b8ac-8921b884916f.ipynb  Users


In [16]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'C:/Users/kelvi/OneDrive/Documents/Data Projects/Data/installments_payments.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

KeyboardInterrupt: 